# 📊 Evaluate Predictions & Settle Ledgers

## Purpose:
- ✅ Fetch actual game results from NBA API
- ✅ Settle predictions in bets_ledger.pkl
- ✅ Calculate win rates, ROI, and calibration
- ✅ Generate calibration curves
- ✅ Recalibrate models with isotonic regression

## What You'll Get:
- Win rate by confidence level
- ROI analysis
- Calibration plots (predicted vs actual)
- Updated calibration.pkl for production use

## Quick Start:
1. Run Setup cell
2. Upload bets_ledger.pkl (your predictions)
3. Run Fetch Results cell
4. Run Evaluation cell
5. Download updated files

---

## 📦 Setup & Installation

In [ ]:
# ============================================================
# SETUP - Install packages and download latest code
# ============================================================

print("="*70)
print("📦 SETUP - Installing packages and pulling latest code")
print("="*70)

print("\n1️⃣ Installing packages...")
!pip install -q nba-api pandas numpy scikit-learn matplotlib seaborn requests
print("   ✅ Packages installed")

print("\n2️⃣ Cleaning up old code...")
import os
import shutil

os.chdir('/content')

# FORCE remove old code to get latest updates
if os.path.exists('meep'):
    shutil.rmtree('meep')
    print("   🧹 Removed old code")

print("\n3️⃣ Cloning latest code from GitHub...")
!git clone -q https://github.com/tyriqmiles0529-pixel/meep.git
os.chdir('meep')
print("   ✅ Code downloaded")

print("\n4️⃣ Verifying code version...")
!git log --oneline -n 3

# Add to Python path
import sys
sys.path.insert(0, '/content/meep')

print("\n" + "="*70)
print("✅ SETUP COMPLETE!")
print("="*70)
print("\n📍 Code version:")
!git log --oneline -n 1
print("\n✅ Latest fixes:")
print("   ✓ Player name matching function added (inline)")
print("   ✓ Ledger path fixed (copies to correct location)")
print("   ✓ No more missing module errors")
print("\nNext: Upload bets_ledger.pkl")

---

## 📤 Upload Bets Ledger

In [ ]:
# ============================================================
# UPLOAD - Bets ledger with predictions to settle
# ============================================================

from google.colab import files
import os
import pickle
import glob

os.chdir('/content')

print("="*70)
print("📤 UPLOAD BETS LEDGER")
print("="*70)
print("\nUpload bets_ledger.pkl (your prediction history)")
print("This file contains all predictions that need to be settled\n")

uploaded = files.upload()

# Handle both bets_ledger.pkl and bets_ledger (1).pkl
ledger_file = None
if 'bets_ledger.pkl' in uploaded:
    ledger_file = 'bets_ledger.pkl'
else:
    # Check for renamed versions
    matches = glob.glob('bets_ledger*.pkl')
    if matches:
        ledger_file = matches[0]
        # Rename to standard name
        os.rename(ledger_file, 'bets_ledger.pkl')
        ledger_file = 'bets_ledger.pkl'
        print(f"\n✅ Renamed {matches[0]} → bets_ledger.pkl")

if ledger_file:
    # Load and inspect
    with open('bets_ledger.pkl', 'rb') as f:
        ledger = pickle.load(f)
    
    bets = ledger.get('bets', [])
    
    if not bets:
        print("\n⚠️  Ledger is empty! No predictions to settle.")
    else:
        total = len(bets)
        settled = sum(1 for bet in bets if bet.get('outcome') in ['won', 'lost'])
        pending = total - settled
        
        print(f"\n✅ Ledger loaded: {total} total predictions")
        print(f"   - {settled} already settled")
        print(f"   - {pending} pending settlement")
        
        if pending == 0:
            print("\n⚠️  All predictions already settled!")
        else:
            print("\n📊 Ready to settle predictions!")
else:
    print("\n❌ No bets_ledger.pkl uploaded")
    print("Please upload your bets ledger file")

---

## 🔄 Fetch Results & Settle Bets

In [ ]:
# ============================================================
# FETCH RESULTS - Get actual outcomes from NBA API
# ============================================================

import pickle
import sys
import os
import shutil

print("="*70)
print("🔄 FETCHING RESULTS & SETTLING BETS")
print("="*70)

# Delete fetch_progress.json to re-settle everything with the bug fix
if os.path.exists('/content/fetch_progress.json'):
    os.remove('/content/fetch_progress.json')
    print("\n🧹 Cleared fetch progress (will re-settle all bets)")
    
if os.path.exists('/content/meep/fetch_progress.json'):
    os.remove('/content/meep/fetch_progress.json')
    print("🧹 Cleared meep fetch progress")

# Copy ledger to meep directory where script expects it
if os.path.exists('/content/bets_ledger.pkl'):
    shutil.copy('/content/bets_ledger.pkl', '/content/meep/bets_ledger.pkl')
    print("\n✅ Ledger copied to /content/meep/")
else:
    print("\n❌ bets_ledger.pkl not found in /content/")
    print("Please run the Upload cell first!")
    raise FileNotFoundError("bets_ledger.pkl not uploaded")

os.chdir('/content/meep')
sys.path.insert(0, '/content/meep')

# Load ledger to show stats
with open('bets_ledger.pkl', 'rb') as f:
    ledger = pickle.load(f)

bets = ledger.get('bets', [])
print(f"\nLoaded {len(bets)} predictions")

# Run the settlement script by executing it as __main__
print("\n⏳ Fetching game results from NBA API...")
print("This may take 1-2 minutes...\n")
print("="*70)

try:
    # Execute the script directly
    exec(open('fetch_bet_results_incremental.py').read())
    
    print("\n" + "="*70)
    print("✅ Settlement complete!")
    print("="*70)
    
except Exception as e:
    print(f"\n❌ Error during settlement: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Possible issues:")
    print("  - NBA API rate limits")
    print("  - Network connectivity")
    print("  - Games still in progress")

---

## 📊 Evaluate Performance

In [ ]:
# ============================================================
# EVALUATE - Calculate metrics and generate reports
# ============================================================

import pickle
import pandas as pd
import numpy as np
from collections import defaultdict

print("="*70)
print("📊 PERFORMANCE EVALUATION")
print("="*70)

# Load settled ledger
with open('/content/meep/bets_ledger.pkl', 'rb') as f:
    ledger = pickle.load(f)

# Filter settled bets - look for 'settled' field, not 'outcome'
settled = [bet for bet in ledger.get('bets', []) if bet.get('settled') == True]

if len(settled) == 0:
    print("\n⚠️  No settled bets found!")
    print("Either all bets are pending or settlement failed.")
    print("\nTroubleshooting:")
    print("  1. Did you run the 'Fetch Results' cell?")
    print("  2. Have the games been played yet? (must be 3+ hours after game time)")
    print("  3. Check if fetch script found any matches")
else:
    print(f"\n✅ Analyzing {len(settled)} settled predictions\n")
    
    # Overall metrics - use 'won' field instead of 'outcome'
    won = sum(1 for bet in settled if bet.get('won') == True)
    lost = len(settled) - won
    win_rate = (won / len(settled)) * 100
    
    print("="*70)
    print("OVERALL RESULTS")
    print("="*70)
    print(f"\nTotal Settled: {len(settled)}")
    print(f"Won: {won}")
    print(f"Lost: {lost}")
    print(f"Win Rate: {win_rate:.2f}%")
    
    # ROI calculation
    total_staked = sum(bet.get('stake', 0) for bet in settled)
    # Calculate profit using actual outcomes
    total_profit = 0
    for bet in settled:
        if bet.get('won'):
            total_profit += bet.get('potential_profit', 0)
        else:
            total_profit -= bet.get('stake', 0)
    
    roi = (total_profit / total_staked * 100) if total_staked > 0 else 0
    
    print(f"\nTotal Staked: ${total_staked:.2f}")
    print(f"Total Profit: ${total_profit:+.2f}")
    print(f"ROI: {roi:+.2f}%")
    
    # By stat type
    print("\n" + "="*70)
    print("BY STAT TYPE")
    print("="*70)
    
    by_stat = defaultdict(lambda: {'won': 0, 'lost': 0, 'staked': 0, 'profit': 0})
    
    for bet in settled:
        stat = bet.get('prop_type', 'unknown')
        by_stat[stat]['won' if bet.get('won') else 'lost'] += 1
        by_stat[stat]['staked'] += bet.get('stake', 0)
        if bet.get('won'):
            by_stat[stat]['profit'] += bet.get('potential_profit', 0)
        else:
            by_stat[stat]['profit'] -= bet.get('stake', 0)
    
    for stat, data in sorted(by_stat.items()):
        total = data['won'] + data['lost']
        wr = (data['won'] / total * 100) if total > 0 else 0
        stat_roi = (data['profit'] / data['staked'] * 100) if data['staked'] > 0 else 0
        print(f"\n{stat.upper()}:")
        print(f"  Win Rate: {wr:.1f}% ({data['won']}-{data['lost']})")
        print(f"  ROI: {stat_roi:+.1f}%")
    
    # By confidence level
    print("\n" + "="*70)
    print("BY CONFIDENCE LEVEL")
    print("="*70)
    
    by_conf = defaultdict(lambda: {'won': 0, 'lost': 0, 'predicted': []})
    
    for bet in settled:
        prob = bet.get('win_prob', 50)
        bucket = int(prob // 10) * 10
        by_conf[bucket]['won' if bet.get('won') else 'lost'] += 1
        by_conf[bucket]['predicted'].append(prob)
    
    print("\nPredicted% | Actual% | Count | Calibration")
    print("-" * 70)
    
    for bucket in sorted(by_conf.keys()):
        data = by_conf[bucket]
        total = data['won'] + data['lost']
        actual = (data['won'] / total * 100) if total > 0 else 0
        predicted = np.mean(data['predicted']) if data['predicted'] else 0
        diff = actual - predicted
        
        calibration = "✅" if abs(diff) < 5 else "⚠️" if abs(diff) < 10 else "❌"
        
        print(f"{bucket}-{bucket+10}%  | {actual:5.1f}% | {total:5d} | {diff:+5.1f}% {calibration}")
    
    print("\n" + "="*70)
    print("CALIBRATION SUMMARY")
    print("="*70)
    print("\n✅ = Well calibrated (within 5%)")
    print("⚠️  = Needs adjustment (5-10% off)")
    print("❌ = Poorly calibrated (>10% off)")

---

## 📈 Generate Calibration Plots

In [ ]:
# ============================================================
# VISUALIZATION - Calibration curves
# ============================================================

import pickle
import numpy as np
import matplotlib.pyplot as plt

print("📈 Generating calibration plots...\n")

# Load settled ledger
with open('/content/meep/bets_ledger.pkl', 'rb') as f:
    ledger = pickle.load(f)

# Use 'settled' field instead of 'outcome'
settled = [bet for bet in ledger.get('bets', []) if bet.get('settled') == True]

if len(settled) == 0:
    print("❌ No settled bets to visualize")
else:
    # Prepare data - use 'won' field instead of 'outcome'
    predicted_probs = np.array([bet.get('win_prob', 50) / 100 for bet in settled])
    actual_outcomes = np.array([1 if bet.get('won') else 0 for bet in settled])
    
    # Create calibration curve
    bins = np.linspace(0, 1, 11)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    bin_actual = []
    bin_counts = []
    
    for i in range(len(bins) - 1):
        mask = (predicted_probs >= bins[i]) & (predicted_probs < bins[i+1])
        if mask.sum() > 0:
            bin_actual.append(actual_outcomes[mask].mean())
            bin_counts.append(mask.sum())
        else:
            bin_actual.append(np.nan)
            bin_counts.append(0)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration', linewidth=2)
    plt.plot(bin_centers, bin_actual, 'o-', linewidth=2, markersize=8, label='Model')
    
    # Add counts
    for x, y, count in zip(bin_centers, bin_actual, bin_counts):
        if not np.isnan(y) and count > 0:
            plt.text(x, y + 0.02, f'n={count}', ha='center', fontsize=9)
    
    plt.xlabel('Predicted Probability', fontsize=12)
    plt.ylabel('Actual Win Rate', fontsize=12)
    plt.title('Model Calibration Curve', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(-0.05, 1.05)
    plt.ylim(-0.05, 1.05)
    plt.tight_layout()
    
    plt.savefig('/content/calibration_analysis.png', dpi=300, bbox_inches='tight')
    print("✅ Calibration plot saved to: calibration_analysis.png")
    plt.show()
    
    print("\n📊 Interpretation:")
    print("  Points on diagonal = perfectly calibrated")
    print("  Points above diagonal = model underconfident")
    print("  Points below diagonal = model overconfident")

---

## 🔧 Recalibrate Models

In [ ]:
# ============================================================
# RECALIBRATE - Train isotonic regression on settled bets
# ============================================================

import pickle
import numpy as np
from sklearn.isotonic import IsotonicRegression
import matplotlib.pyplot as plt

print("="*70)
print("🔧 RECALIBRATING MODELS")
print("="*70)

# Load settled ledger
with open('/content/meep/bets_ledger.pkl', 'rb') as f:
    ledger = pickle.load(f)

# Use 'settled' field instead of 'outcome'
settled = [bet for bet in ledger.get('bets', []) if bet.get('settled') == True]

if len(settled) < 30:
    print(f"\n⚠️  Only {len(settled)} settled bets - need at least 30 for calibration")
    print("Keep settling more bets and run this again later")
else:
    print(f"\n✅ Using {len(settled)} settled bets for calibration\n")
    
    # Prepare data by stat type
    calibrators = {}
    
    stat_types = set(bet.get('prop_type', 'unknown') for bet in settled)
    
    for stat_type in stat_types:
        stat_bets = [bet for bet in settled if bet.get('prop_type') == stat_type]
        
        if len(stat_bets) >= 20:  # Need enough samples
            X = np.array([bet.get('win_prob', 50) / 100 for bet in stat_bets])
            # Use 'won' field instead of 'outcome'
            y = np.array([1 if bet.get('won') else 0 for bet in stat_bets])
            
            # Train isotonic regression
            iso = IsotonicRegression(out_of_bounds='clip')
            iso.fit(X, y)
            
            calibrators[stat_type] = iso
            
            print(f"✅ Calibrated {stat_type.upper()} ({len(stat_bets)} bets)")
        else:
            print(f"⚠️  Skipped {stat_type.upper()} (only {len(stat_bets)} bets, need 20+)")
    
    # Save calibrators
    calibration_file = '/content/calibration.pkl'
    with open(calibration_file, 'wb') as f:
        pickle.dump(calibrators, f)
    
    print(f"\n✅ Calibration saved to: {calibration_file}")
    print("\nDownload this file and use it with riq_analyzer.py for better predictions!")
    
    # Visualize calibration curves
    if calibrators:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        x_range = np.linspace(0, 1, 100)
        
        ax.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration', linewidth=2)
        
        for stat_type, iso in calibrators.items():
            y_calibrated = iso.predict(x_range)
            ax.plot(x_range, y_calibrated, label=stat_type.upper(), linewidth=2, alpha=0.7)
        
        ax.set_xlabel('Original Predicted Probability', fontsize=12)
        ax.set_ylabel('Calibrated Probability', fontsize=12)
        ax.set_title('Isotonic Calibration Curves by Stat Type', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        
        plt.tight_layout()
        plt.savefig('/content/calibration_curves.png', dpi=300, bbox_inches='tight')
        print("\n📊 Calibration curves saved to: calibration_curves.png")
        plt.show()

---

## 💾 Download Updated Files

In [ ]:
# ============================================================
# DOWNLOAD - Get updated files to use in production
# ============================================================

from google.colab import files
import os

print("="*70)
print("💾 DOWNLOAD UPDATED FILES")
print("="*70)

print("\nDownloading files...\n")

# Download settled ledger
if os.path.exists('/content/meep/bets_ledger.pkl'):
    files.download('/content/meep/bets_ledger.pkl')
    print("✅ Downloaded: bets_ledger.pkl (updated with results)")

# Download calibration
if os.path.exists('/content/calibration.pkl'):
    files.download('/content/calibration.pkl')
    print("✅ Downloaded: calibration.pkl (use with riq_analyzer.py)")

# Download plots
if os.path.exists('/content/calibration_analysis.png'):
    files.download('/content/calibration_analysis.png')
    print("✅ Downloaded: calibration_analysis.png")

if os.path.exists('/content/calibration_curves.png'):
    files.download('/content/calibration_curves.png')
    print("✅ Downloaded: calibration_curves.png")

print("\n" + "="*70)
print("✅ ALL FILES DOWNLOADED!")
print("="*70)
print("\nWhat to do next:")
print("  1. Copy calibration.pkl to your project root")
print("  2. riq_analyzer.py will auto-load it on next run")
print("  3. Predictions will now be better calibrated!")
print("\nRe-run this notebook weekly to keep improving calibration.")

---

## 📝 Generate Summary Report

In [ ]:
# ============================================================# SUMMARY REPORT - Text file with all metrics# ============================================================import pickleimport numpy as npfrom datetime import datetimefrom collections import defaultdictprint("📝 Generating summary report...\n")# Load settled ledgerwith open('/content/meep/bets_ledger.pkl', 'rb') as f:    ledger = pickle.load(f)settled = [bet for bet in ledger.get('bets', []) if bet.get('outcome') in ['won', 'lost']]pending = [bet for bet in ledger.get('bets', []) if bet.get('outcome') not in ['won', 'lost']]if len(settled) > 0:    # Generate report    report = []    report.append("="*80)    report.append("NBA PREDICTION EVALUATION REPORT")    report.append("="*80)    report.append(f"\nGenerated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")    report.append(f"Total Predictions: {len(ledger)}")    report.append(f"Settled: {len(settled)}")    report.append(f"Pending: {len(pending)}")        # Overall metrics    won = sum(1 for bet in settled if bet['outcome'] == 'won')    lost = len(settled) - won    win_rate = (won / len(settled)) * 100        total_staked = sum(bet.get('stake', 0) for bet in settled)    total_profit = sum(bet.get('profit', 0) if bet['outcome'] == 'won' else -bet.get('stake', 0) for bet in settled)    roi = (total_profit / total_staked * 100) if total_staked > 0 else 0        report.append("\n" + "="*80)    report.append("OVERALL PERFORMANCE")    report.append("="*80)    report.append(f"\nWin Rate: {win_rate:.2f}% ({won}W-{lost}L)")    report.append(f"Total Staked: ${total_staked:.2f}")    report.append(f"Total Profit: ${total_profit:+.2f}")    report.append(f"ROI: {roi:+.2f}%")        # By stat type    report.append("\n" + "="*80)    report.append("PERFORMANCE BY STAT TYPE")    report.append("="*80)        by_stat = defaultdict(lambda: {'won': 0, 'lost': 0, 'staked': 0, 'profit': 0})        for bet in settled:        stat = bet.get('prop_type', 'unknown')        by_stat[stat]['won' if bet['outcome'] == 'won' else 'lost'] += 1        by_stat[stat]['staked'] += bet.get('stake', 0)        if bet['outcome'] == 'won':            by_stat[stat]['profit'] += bet.get('profit', 0)        else:            by_stat[stat]['profit'] -= bet.get('stake', 0)        for stat, data in sorted(by_stat.items()):        total = data['won'] + data['lost']        wr = (data['won'] / total * 100) if total > 0 else 0        stat_roi = (data['profit'] / data['staked'] * 100) if data['staked'] > 0 else 0        report.append(f"\n{stat.upper()}:")        report.append(f"  Record: {data['won']}-{data['lost']} ({wr:.1f}%)")        report.append(f"  Staked: ${data['staked']:.2f}")        report.append(f"  Profit: ${data['profit']:+.2f}")        report.append(f"  ROI: {stat_roi:+.1f}%")        # Calibration    report.append("\n" + "="*80)    report.append("CALIBRATION ANALYSIS")    report.append("="*80)        by_conf = defaultdict(lambda: {'won': 0, 'lost': 0, 'predicted': []})        for bet in settled:        prob = bet.get('win_prob', 50)        bucket = int(prob // 10) * 10        by_conf[bucket]['won' if bet['outcome'] == 'won' else 'lost'] += 1        by_conf[bucket]['predicted'].append(prob)        report.append("\nPredicted% | Actual%  | Count | Calibration")    report.append("-" * 80)        for bucket in sorted(by_conf.keys()):        data = by_conf[bucket]        total = data['won'] + data['lost']        actual = (data['won'] / total * 100) if total > 0 else 0        predicted = np.mean(data['predicted']) if data['predicted'] else 0        diff = actual - predicted                if abs(diff) < 5:            cal = "Well calibrated"        elif abs(diff) < 10:            cal = "Needs adjustment"        else:            cal = "Poorly calibrated"                report.append(f"{bucket:3d}-{bucket+10:3d}% | {actual:6.1f}% | {total:5d} | {diff:+5.1f}% - {cal}")        report.append("\n" + "="*80)    report.append("END OF REPORT")    report.append("="*80)        # Save report    report_text = "\n".join(report)        with open('/content/evaluation_report.txt', 'w') as f:        f.write(report_text)        print(report_text)        print("\n\n✅ Report saved to: evaluation_report.txt")        # Download    from google.colab import files    files.download('/content/evaluation_report.txt')    print("📥 Downloading report...")else:    print("⚠️  No settled bets to report on")